<a href="https://colab.research.google.com/github/KevinTheRainmaker/AI_PlayGround/blob/main/SMOTE_Trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
#Load Library & Framework
import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
import lightgbm
from lightgbm import LGBMClassifier
import torch.utils.data as data_utils
from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
df = pd.read_csv('drive/MyDrive/creditcard.csv')
# df.Class.value_counts(normalize=True).plot(kind='bar')
print(df.Class.value_counts(normalize=True)*100)

0    99.827251
1     0.172749
Name: Class, dtype: float64


In [19]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=10)

In [22]:
#Metircs
def metrics(y_test,pred):
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    f1 = f1_score(y_test,pred)
    roc_score = roc_auc_score(y_test,pred,average='macro')
    print('Accuracy : {0:.2f}, Precision : {1:.2f}, Recall : {2:.2f}'.format(accuracy,precision,recall))
    print('f1-score : {0:.2f}, auc : {1:.2f}'.format(f1,roc_score,recall))

#Modeling
def modeling(model,x_train,x_test,y_train,y_test):
    model.fit(x_train,y_train)
    pred = model.predict(x_test)
    metrics(y_test,pred)

In [23]:
#SMOTE
smote = SMOTE(random_state=0)
X_train_over,y_train_over = smote.fit_sample(X_train,y_train)
print('Features for train / label Dataset before SMOTE: ', X_train.shape, y_train.shape)
print('Features for train / label Dataset after SMOTE: ', X_train_over.shape, y_train_over.shape)
print('Label value distribution after SMOTE: \n', pd.Series(y_train_over).value_counts())

Features for train / label Dataset before SMOTE:  (213605, 30) (213605,)
Features for train / label Dataset after SMOTE:  (426468, 30) (426468,)
Label value distribution after SMOTE: 
 1    213234
0    213234
dtype: int64


In [24]:
#Logistic
lr = LogisticRegression()
modeling(lr,X_train_over,X_test,y_train_over,y_test)

Accuracy : 0.98, Precision : 0.08, Recall : 0.91
f1-score : 0.15, auc : 0.95


In [25]:
#lightgbm
lgb = LGBMClassifier(n_estimators=1000,num_leaves=64,n_jobs=-1,boost_from_average=False)
modeling(lgb,X_train_over,X_test,y_train_over,y_test)

Accuracy : 1.00, Precision : 0.89, Recall : 0.85
f1-score : 0.87, auc : 0.93
